In [3]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [4]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [5]:
cleanText('hi \n\t')

'hi'

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
model_ner = spacy.load('./output/model-best/')

In [9]:
print(model_ner)

In [10]:
# Load Image
image = cv2.imread('./data/6.jpg')

# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [11]:
doc

cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo

In [12]:
from spacy import displacy

In [13]:
displacy.render(doc,style='ent')

In [14]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [15]:
doc_text = docjson['text']

In [18]:
doc_text

'cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo'

In [21]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,ga
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [22]:
datafram_tokens

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,ga
3,3,19,29,8466045457
4,4,30,35,email
...,...,...,...,...
58,58,448,455,promote
59,59,456,460,your
60,60,461,469,business
61,61,470,476,online


In [23]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [24]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,ga,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [25]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [26]:
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [27]:
dataframe_info

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,3,1,1,1,722,53,64,28,90.167198,cell,4,0,cell,O
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,B-PHONE
2,5,1,3,2,1,1,55,55,85,89,47.185547,ga,18,16,ga,O
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,B-PHONE
4,5,1,3,2,2,1,593,136,93,25,89.289780,email,35,30,email,O
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,B-ORG
7,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63,insurance,I-ORG
8,5,1,3,3,1,3,183,170,117,14,96.909729,corporation,84,73,corporation,I-ORG
9,5,1,3,3,1,4,306,170,20,14,96.172005,of,87,85,of,I-ORG
